In [1]:
# 분석 목적
# 1. 미세먼지가 많은 날 센서 값에 어떤 영향을 주는가

In [30]:
import pandas as pd
from datetime import date
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
import numpy as np
import seaborn as sns


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)

### DB연결

In [44]:
import sys #1,2 위에

import pymysql

# 패키지 임포트 셀 하단에 위치
# 같은 폴더 내에 있으니까 주석해도 가능
# sys.path.append('../') #1
import connect_info_dev #2

In [45]:
# db connect - close
# cursor connect - close
def connect_db(conndb):
    conndb = conndb
    conn = pymysql.connect(host=conndb['host'],
                           user=conndb['user'],
                           password=conndb['password'],
                           port=conndb['port'],
                           charset=conndb['charset'],
                           db=conndb['dbname'])
    return conn

In [46]:
dbconnect = connect_db(connect_info_dev.AirdeepDevDb)
cursor = dbconnect.cursor()

## 1. 심야차량에 흡연하는것

In [34]:
# 데이터 가져오기
qry1 = """
 SELECT A.*
        , B.DETC_CD
        , B.DETC_TIME
  FROM  (
        SELECT  *
          FROM  otoair.OTOAIR_SENSOR_DEVICE
         WHERE  CLIENT_ID = "b1446038ac4a46cc8718a25f14ab4a46"
                AND MSG_DATE BETWEEN '2021-05-01' AND '2021-06-01'
        ) A
  LEFT
  JOIN  (
        SELECT  *
          FROM  otoair.OTOAIR_DETECT_RESULT
         WHERE  CLIENT_ID = "b1446038ac4a46cc8718a25f14ab4a46"
                AND DETC_TIME BETWEEN '2021-05-01' AND '2021-06-01'
        ) B
    ON  A.TERMINAL_ID = B.TERMINAL_ID
        AND  A.MSG_DATE=B.DETC_TIME
 ORDER
    BY  A.TERMINAL_ID, MSG_DATE ASC
"""

In [35]:
cursor.execute(qry1)
result1 = cursor.fetchall()

In [36]:
# 쿼리 실행 및 결과 저장
column = ['MSG_DATE','TERMINAL_ID','PM2_5_MAX','PM10_MAX','TVOC_MAX','ECO2_MAX','REGION_ID','CLIENT_ID','DETC_CD','DETC_TIME']

df_may = pd.DataFrame(result1, columns=column)

In [37]:
df_may

,MSG_DATE,TERMINAL_ID,PM2_5_MAX,PM10_MAX,TVOC_MAX,ECO2_MAX,REGION_ID,CLIENT_ID,DETC_CD,DETC_TIME
0,2021-05-06 10:17:40,0050bc7cad61a77f8bd08ca6c621b5fa,18,50,2,414,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT
1,2021-05-06 10:18:39,0050bc7cad61a77f8bd08ca6c621b5fa,1,1,12,480,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT
2,2021-05-06 10:19:39,0050bc7cad61a77f8bd08ca6c621b5fa,0,1,3,422,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT
3,2021-05-06 10:20:39,0050bc7cad61a77f8bd08ca6c621b5fa,0,1,1,411,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT
4,2021-05-06 10:21:39,0050bc7cad61a77f8bd08ca6c621b5fa,0,2,4,432,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT
...,...,...,...,...,...,...,...,...,...,...
420366,2021-05-31 23:38:49,f9edd24b2cde19e3b9775c8fdfd42be3,15,17,13,487,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT
420367,2021-05-31 23:39:19,f9edd24b2cde19e3b9775c8fdfd42be3,16,18,6,441,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT
420368,2021-05-31 23:39:49,f9edd24b2cde19e3b9775c8fdfd42be3,15,16,5,438,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT
420369,2021-05-31 23:40:19,f9edd24b2cde19e3b9775c8fdfd42be3,21,22,5,433,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT


In [10]:
# CSV 저장
df_may.to_csv('df_may.csv',header=True, index=False)

In [47]:
# 데이터 가져오기
qry2 = """
SELECT  MSRDT
        , DATE_FORMAT(MSRDT, '%Y-%m-%d') AS M_DATE 
        , HOUR(DATE_ADD(MSRDT, INTERVAL -1 HOUR )) AS M_HOUR
        , PM10
        , PM25
  FROM  SEOUL_DUST sd 
 WHERE  MSRDT BETWEEN '21-04-30' AND '21-06-02'
"""

In [48]:
cursor.execute(qry2)
result2 = cursor.fetchall()

In [49]:
# 쿼리 실행 및 결과 저장
column = ['MSRDT','M_DATE','M_HOUR','PM10','PM25']

df_may_dust = pd.DataFrame(result2, columns=column)

In [50]:
df_may_dust

,MSRDT,M_DATE,M_HOUR,PM10,PM25
0,2021-05-10 13:05:01,2021-05-10,12,18.0,7.0
1,2021-05-10 12:05:00,2021-05-10,11,20.0,8.0
2,2021-05-10 11:05:01,2021-05-10,10,21.0,8.0
3,2021-05-10 10:07:16,2021-05-10,9,23.0,9.0
4,2021-05-10 10:05:01,2021-05-10,9,23.0,9.0
...,...,...,...,...,...
1301,2021-06-01 19:05:01,2021-06-01,18,37.0,22.0
1302,2021-06-01 20:05:01,2021-06-01,19,39.0,24.0
1303,2021-06-01 21:05:00,2021-06-01,20,42.0,26.0
1304,2021-06-01 22:05:00,2021-06-01,21,47.0,28.0


In [51]:
# CSV 저장
df_may_dust.to_csv('df_may_dust.csv',header=True, index=False)

##### SQL로 불러오고 난 후 아래 꼭 하기

In [52]:
cursor.close()
dbconnect.close()

#### 분석 시작(1)

In [75]:
df_may['HOUR'] = df_may['MSG_DATE'].dt.hour
df_may

,MSG_DATE,TERMINAL_ID,PM2_5_MAX,PM10_MAX,TVOC_MAX,ECO2_MAX,REGION_ID,CLIENT_ID,DETC_CD,DETC_TIME,HOUR
0,2021-05-06 10:17:40,0050bc7cad61a77f8bd08ca6c621b5fa,18,50,2,414,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,10
1,2021-05-06 10:18:39,0050bc7cad61a77f8bd08ca6c621b5fa,1,1,12,480,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,10
2,2021-05-06 10:19:39,0050bc7cad61a77f8bd08ca6c621b5fa,0,1,3,422,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,10
3,2021-05-06 10:20:39,0050bc7cad61a77f8bd08ca6c621b5fa,0,1,1,411,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,10
4,2021-05-06 10:21:39,0050bc7cad61a77f8bd08ca6c621b5fa,0,2,4,432,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,10
...,...,...,...,...,...,...,...,...,...,...,...
420366,2021-05-31 23:38:49,f9edd24b2cde19e3b9775c8fdfd42be3,15,17,13,487,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,23
420367,2021-05-31 23:39:19,f9edd24b2cde19e3b9775c8fdfd42be3,16,18,6,441,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,23
420368,2021-05-31 23:39:49,f9edd24b2cde19e3b9775c8fdfd42be3,15,16,5,438,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,23
420369,2021-05-31 23:40:19,f9edd24b2cde19e3b9775c8fdfd42be3,21,22,5,433,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,23


In [80]:
# TERMINAL_ID와 MSG_DATE로 정렬
df_may_sort = df_may.sort_values(['TERMINAL_ID', 'MSG_DATE'], ascending=[True, True])

# 날짜 형식으로 변경한 새로운 'MSG_DATE2' 생성
df_may_sort['MSG_DATE2']= df_may_sort['MSG_DATE'].astype('datetime64[ns]')

# 로그 사이의 시간 계산
df_may_sort['DATE_DIFF']=(df_may_sort['MSG_DATE2']-df_may_sort['MSG_DATE2'].shift(1)).dt.total_seconds()

# 처음 정렬 된 0050bc7cad61a77f8bd08ca6c621b5fa	 차량은 PERIOD 가 60이므로 Nan 값을 60으로 대체
df_may_sort['DATE_DIFF'] = df_may_sort['DATE_DIFF'].fillna(60)

# index 초기화()
df_may_01 = df_may_sort.reset_index(drop=True)

In [81]:
# TERMINAL_ID가 달라지거나
# 300초(5분) 동안 데이터가 수집이 되지 않으면 이용하는 고객이 변한다고 가정(보수적으로 하기 위해서 300초 이상도 가능)
# C_NUM = 고객 번호 / INDEX 수집 되는 데이터 순서
aaa = [0]
bbb = 0
ccc = []
for i in range(1,len(df_may_01)):
    if df_may_01['TERMINAL_ID'][i] != df_may_01['TERMINAL_ID'][i-1] or df_may_01['DATE_DIFF'][i] >=300:
        bbb = bbb+1
    else:
        bbb = bbb
    aaa.append(bbb)

df_may_01['C_NUM'] = aaa


for i in range(0,bbb+1):
    for j in range(len(df_may_01[df_may_01['C_NUM']==i])):
        ccc.append(j+1)
        
df_may_01['INDEX'] = ccc     

In [82]:
df_may_01

,MSG_DATE,TERMINAL_ID,PM2_5_MAX,PM10_MAX,TVOC_MAX,ECO2_MAX,REGION_ID,CLIENT_ID,DETC_CD,DETC_TIME,HOUR,MSG_DATE2,DATE_DIFF,C_NUM,INDEX
0,2021-05-06 10:17:40,0050bc7cad61a77f8bd08ca6c621b5fa,18,50,2,414,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,10,2021-05-06 10:17:40,60.0,0,1
1,2021-05-06 10:18:39,0050bc7cad61a77f8bd08ca6c621b5fa,1,1,12,480,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,10,2021-05-06 10:18:39,59.0,0,2
2,2021-05-06 10:19:39,0050bc7cad61a77f8bd08ca6c621b5fa,0,1,3,422,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,10,2021-05-06 10:19:39,60.0,0,3
3,2021-05-06 10:20:39,0050bc7cad61a77f8bd08ca6c621b5fa,0,1,1,411,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,10,2021-05-06 10:20:39,60.0,0,4
4,2021-05-06 10:21:39,0050bc7cad61a77f8bd08ca6c621b5fa,0,2,4,432,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,10,2021-05-06 10:21:39,60.0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420366,2021-05-31 23:38:49,f9edd24b2cde19e3b9775c8fdfd42be3,15,17,13,487,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,23,2021-05-31 23:38:49,30.0,4020,249
420367,2021-05-31 23:39:19,f9edd24b2cde19e3b9775c8fdfd42be3,16,18,6,441,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,23,2021-05-31 23:39:19,30.0,4020,250
420368,2021-05-31 23:39:49,f9edd24b2cde19e3b9775c8fdfd42be3,15,16,5,438,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,23,2021-05-31 23:39:49,30.0,4020,251
420369,2021-05-31 23:40:19,f9edd24b2cde19e3b9775c8fdfd42be3,21,22,5,433,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,23,2021-05-31 23:40:19,30.0,4020,252


In [95]:
# CON_NON_SMOKE = 
C_NUM_LIST = df_may_01['C_NUM'].unique()
C_NUM_N_SMOKE = df_may_01[(df_may_01['DETC_CD'] == 'C110') | (df_may_01['DETC_CD'] == 'C120') ]['C_NUM'].unique()

main_list = list(set(C_NUM_LIST) - set(C_NUM_N_SMOKE))

In [96]:
main_list = list(set(C_NUM_LIST) - set(C_NUM_N_SMOKE))

#### 분석 시작(3)

In [5]:
## csv 파일로 가져온 것

# 210501-210516_log의 전체 log를 사용
#df2=pd.read_csv("C:/Users/hy.kim/Desktop/건욱/AirDeep/심야차량/(그린카)심야차량log_210501-210516_log.csv")
#df2

,MSG_DATE,TERMINAL_ID,PM2_5_MAX,PM10_MAX,TVOC_MAX,ECO2_MAX,REGION_ID,CLIENT_ID,HOUR,DETC_CD,DETC_TIME
0,2021-05-06 10:17:40,0050bc7cad61a77f8bd08ca6c621b5fa,18,50,2,414,test-qa,b1446038ac4a46cc8718a25f14ab4a46,10H,NaN,NaN
1,2021-05-06 10:18:39,0050bc7cad61a77f8bd08ca6c621b5fa,1,1,12,480,test-qa,b1446038ac4a46cc8718a25f14ab4a46,10H,NaN,NaN
2,2021-05-06 10:19:39,0050bc7cad61a77f8bd08ca6c621b5fa,0,1,3,422,test-qa,b1446038ac4a46cc8718a25f14ab4a46,10H,NaN,NaN
3,2021-05-06 10:20:39,0050bc7cad61a77f8bd08ca6c621b5fa,0,1,1,411,test-qa,b1446038ac4a46cc8718a25f14ab4a46,10H,NaN,NaN
4,2021-05-06 10:21:39,0050bc7cad61a77f8bd08ca6c621b5fa,0,2,4,432,test-qa,b1446038ac4a46cc8718a25f14ab4a46,10H,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
225492,2021-05-16 23:57:30,f9edd24b2cde19e3b9775c8fdfd42be3,0,0,2,417,test-qa,b1446038ac4a46cc8718a25f14ab4a46,23H,NaN,NaN
225493,2021-05-16 23:58:00,f9edd24b2cde19e3b9775c8fdfd42be3,0,0,5,435,test-qa,b1446038ac4a46cc8718a25f14ab4a46,23H,NaN,NaN
225494,2021-05-16 23:58:30,f9edd24b2cde19e3b9775c8fdfd42be3,0,0,7,452,test-qa,b1446038ac4a46cc8718a25f14ab4a46,23H,NaN,NaN
225495,2021-05-16 23:59:00,f9edd24b2cde19e3b9775c8fdfd42be3,0,0,9,465,test-qa,b1446038ac4a46cc8718a25f14ab4a46,23H,NaN,NaN


In [20]:
# TERMINAL_ID와 MSG_DATE로 정렬
df3_2 = df2.sort_values(['TERMINAL_ID', 'MSG_DATE'], ascending=[True, True])

# 날짜 형식으로 변경한 새로운 'MSG_DATE2' 생성
df3_2['MSG_DATE2']= df3_2['MSG_DATE'].astype('datetime64[ns]')

# 로그 사이의 시간 계산
df3_2['DATE_DIFF']=(df3_2['MSG_DATE2']-df3_2['MSG_DATE2'].shift(1)).dt.total_seconds()

# 처음 정렬 된 0050bc7cad61a77f8bd08ca6c621b5fa	 차량은 PERIOD 가 60이므로 Nan 값을 60으로 대체
df3_2['DATE_DIFF'] = df3_2['DATE_DIFF'].fillna(60)

# index 초기화()
df3_2 = df3_2.reset_index(drop=True)

In [21]:
df3_2

,MSG_DATE,TERMINAL_ID,PM2_5_MAX,PM10_MAX,TVOC_MAX,ECO2_MAX,REGION_ID,CLIENT_ID,DETC_CD,DETC_TIME,MSG_DATE2,DATE_DIFF
0,2021-05-06 10:17:40,0050bc7cad61a77f8bd08ca6c621b5fa,18,50,2,414,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-06 10:17:40,60.0
1,2021-05-06 10:18:39,0050bc7cad61a77f8bd08ca6c621b5fa,1,1,12,480,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-06 10:18:39,59.0
2,2021-05-06 10:19:39,0050bc7cad61a77f8bd08ca6c621b5fa,0,1,3,422,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-06 10:19:39,60.0
3,2021-05-06 10:20:39,0050bc7cad61a77f8bd08ca6c621b5fa,0,1,1,411,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-06 10:20:39,60.0
4,2021-05-06 10:21:39,0050bc7cad61a77f8bd08ca6c621b5fa,0,2,4,432,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-06 10:21:39,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...
225492,2021-05-16 23:57:30,f9edd24b2cde19e3b9775c8fdfd42be3,0,0,2,417,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-16 23:57:30,30.0
225493,2021-05-16 23:58:00,f9edd24b2cde19e3b9775c8fdfd42be3,0,0,5,435,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-16 23:58:00,30.0
225494,2021-05-16 23:58:30,f9edd24b2cde19e3b9775c8fdfd42be3,0,0,7,452,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-16 23:58:30,30.0
225495,2021-05-16 23:59:00,f9edd24b2cde19e3b9775c8fdfd42be3,0,0,9,465,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-16 23:59:00,30.0


In [22]:
# TERMINAL_ID가 달라지거나
# 600초(10분) 동안 데이터가 수집이 되지 않으면 이용하는 고객이 변한다고 가정(보수적으로 하기 위해서 600초 이상도 가능)
aaa = [0]
bbb = 0

for i in range(1,len(df2_2)):
    if df3_2['TERMINAL_ID'][i] != df3_2['TERMINAL_ID'][i-1] or df3_2['DATE_DIFF'][i] >=600:
        bbb = bbb+1 
    else:
        bbb = bbb
    aaa.append(bbb)
       
df3_2['C_NUM']  = aaa

In [23]:
df3_2

,MSG_DATE,TERMINAL_ID,PM2_5_MAX,PM10_MAX,TVOC_MAX,ECO2_MAX,REGION_ID,CLIENT_ID,DETC_CD,DETC_TIME,MSG_DATE2,DATE_DIFF,C_NUM
0,2021-05-06 10:17:40,0050bc7cad61a77f8bd08ca6c621b5fa,18,50,2,414,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-06 10:17:40,60.0,0
1,2021-05-06 10:18:39,0050bc7cad61a77f8bd08ca6c621b5fa,1,1,12,480,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-06 10:18:39,59.0,0
2,2021-05-06 10:19:39,0050bc7cad61a77f8bd08ca6c621b5fa,0,1,3,422,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-06 10:19:39,60.0,0
3,2021-05-06 10:20:39,0050bc7cad61a77f8bd08ca6c621b5fa,0,1,1,411,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-06 10:20:39,60.0,0
4,2021-05-06 10:21:39,0050bc7cad61a77f8bd08ca6c621b5fa,0,2,4,432,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-06 10:21:39,60.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
225492,2021-05-16 23:57:30,f9edd24b2cde19e3b9775c8fdfd42be3,0,0,2,417,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-16 23:57:30,30.0,1994
225493,2021-05-16 23:58:00,f9edd24b2cde19e3b9775c8fdfd42be3,0,0,5,435,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-16 23:58:00,30.0,1994
225494,2021-05-16 23:58:30,f9edd24b2cde19e3b9775c8fdfd42be3,0,0,7,452,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-16 23:58:30,30.0,1994
225495,2021-05-16 23:59:00,f9edd24b2cde19e3b9775c8fdfd42be3,0,0,9,465,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-16 23:59:00,30.0,1994


In [12]:
pd.set_option('display.max_rows', 500000)
pd.set_option('display.max_columns', 20)

In [24]:
# C_NUM을 기준으로 날짜를 오름차순으로 정렬하고 상위 5개씩 가져옴
df3_3 = df3_2.sort_values(by="MSG_DATE2", ascending=True).groupby("C_NUM").head(5)
# 정렬
df3_3.sort_values(by=["C_NUM", "MSG_DATE2"], ascending=[True, True])

,MSG_DATE,TERMINAL_ID,PM2_5_MAX,PM10_MAX,TVOC_MAX,ECO2_MAX,REGION_ID,CLIENT_ID,DETC_CD,DETC_TIME,MSG_DATE2,DATE_DIFF,C_NUM
0,2021-05-06 10:17:40,0050bc7cad61a77f8bd08ca6c621b5fa,18,50,2,414,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-06 10:17:40,60.0,0
1,2021-05-06 10:18:39,0050bc7cad61a77f8bd08ca6c621b5fa,1,1,12,480,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-06 10:18:39,59.0,0
2,2021-05-06 10:19:39,0050bc7cad61a77f8bd08ca6c621b5fa,0,1,3,422,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-06 10:19:39,60.0,0
3,2021-05-06 10:20:39,0050bc7cad61a77f8bd08ca6c621b5fa,0,1,1,411,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-06 10:20:39,60.0,0
4,2021-05-06 10:21:39,0050bc7cad61a77f8bd08ca6c621b5fa,0,2,4,432,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-06 10:21:39,60.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
225378,2021-05-16 22:58:55,f9edd24b2cde19e3b9775c8fdfd42be3,44,62,0,403,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-16 22:58:55,653.0,1994
225379,2021-05-16 22:59:56,f9edd24b2cde19e3b9775c8fdfd42be3,25,25,2,415,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-16 22:59:56,61.0,1994
225380,2021-05-16 23:00:26,f9edd24b2cde19e3b9775c8fdfd42be3,19,19,4,428,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-16 23:00:26,30.0,1994
225381,2021-05-16 23:00:57,f9edd24b2cde19e3b9775c8fdfd42be3,11,11,8,459,test-qa,b1446038ac4a46cc8718a25f14ab4a46,None,NaT,2021-05-16 23:00:57,31.0,1994
